### Piping a Prompt, Model and an Output Parser


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

In [4]:
list_instructions = CommaSeparatedListOutputParser().get_format_instructions()

In [5]:
list_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [ ]:
chat_template = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            "I have recently adopted a {pet}. Can you provide me with a list of three unique names for my new {pet}? \n"
            + list_instructions,
        ),
    ]
)

In [14]:
chat_template

ChatPromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, template='I have recently adopted a {pet}. Can you provide me with a list of three unique names for my new {pet}? \nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'), additional_kwargs={})])

In [15]:
print(chat_template.messages[0].prompt.template)

I have recently adopted a {pet}. Can you provide me with a list of three unique names for my new {pet}? 
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [16]:
chat = ChatOpenAI(
    api_key=api_key,
    model_name="gpt-4.1-mini",
    temperature=0,
    max_tokens=100,
    seed=42,
)

In [17]:
list_output_parser = CommaSeparatedListOutputParser()

In [18]:
chat_template_result = chat_template.invoke({"pet": "dog"})

In [19]:
chat_result = chat.invoke(chat_template_result)

In [22]:
list_output_parser.invoke(chat_result)

['Nimbus', 'Zephyr', 'Koda']

In [23]:
chain = chat_template | chat | list_output_parser

In [24]:
chain

ChatPromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, template='I have recently adopted a {pet}. Can you provide me with a list of three unique names for my new {pet}? \nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x117df4350>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x117e5ab90>, root_client=<openai.OpenAI object at 0x11793ae50>, root_async_client=<openai.AsyncOpenAI object at 0x120f01810>, model_name='gpt-4.1-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True, seed=42, max_tokens=100)
| CommaSeparatedListOutputParser()

In [25]:
chain.invoke({"pet": "cat"})

['Nimbus', 'Zephyr', 'Miso']